### Deep Q-learning

Värdebaserade metoder lär sig förutsäga värden istället för att direkt optimera på en loss-funktion. I de flesta fall används ett _Q värde_, där Q antyder "quality". I allmänhet innebär det att vi måste ha någon sorts data att först lära oss Q-värdena från. En fortfarande använd, men ineffektiv, metod är att bygga upp ett stickprov genom att observera utfallet av slumpmässiga handlingar under någon viss tid. Alternativ kan förstås en sådan _erfarenhetsbuffer_ förberedas med kända exempel, som "alla partier turneringsschack spelade sedan 1886". 

Under ytan handlar det egentligen om en statistisk modell över _stokastiska processer_ även kända som _Markov Decision Processes_ (MDP). Se kapitel 19 i boken för mer detaljer.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import gymnasium as gym
max_episode_steps=1000
def show_one_episode(policy, seed=42):
    frames = []
    env = gym.make("CartPole-v1", render_mode="human",
                   max_episode_steps=max_episode_steps)
    obs, info = env.reset(seed=seed)
    while True:
        frames.append(env.render())
        action = policy(obs)
        obs, reward, done, truncated, info = env.step(action)
        if done or truncated:
            print(f"Failed? {done}")
            break        
    env.close()

DQN nätverket lär sig förutsäga Q-värdena givet ett visst tillstånd. För att välja en handling tar vi helt enkelt den med högst förutsagt värde. 'Epsilon' värdet, en hyperparameter för metoden, anger sannolikheten att vi tar en slumpmässig handling.

In [55]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(4, 32), nn.ReLU(),
                                 nn.Linear(32, 32), nn.ReLU(),
                                 nn.Linear(32, 2))

    def forward(self, state):
        return self.net(state)

def choose_dqn_action(model, obs, epsilon=0.0):
        if torch.rand(()) < epsilon:  # epsilon greedy policy
            return torch.randint(2, size=()).item()
        else:
            state = torch.as_tensor(obs)
            Q_values = model(state)
            return Q_values.argmax().item()  # optimal according to the DQN

Givet en _replay buffer_ tar vi slumpmässiga stickprov från den, <code>batch_size</code> långa. Buffern är ordnad över tid, så det är bara startpunkten som är slumpmässig-- stickprovet är en serie tillstånd med tillhörande handlingar.

In [56]:
def sample_experiences(replay_buffer, batch_size):
    indices = torch.randint(len(replay_buffer), size=[batch_size])
    batch = [replay_buffer[index] for index in indices.tolist()]
    return [to_tensor([exp[index] for exp in batch]) for index in range(6)]

def to_tensor(data):
    array = np.stack(data)
    dtype = torch.float32 if array.dtype == np.float64 else None
    return torch.as_tensor(array, dtype=dtype)

Ett sätt att implementera en fix buffer:

In [57]:
class ReplayBuffer:
    def __init__(self, max_length):
        self.data = [None] * max_length
        self.max_length = max_length
        self.index = 0
        self.length = 0

    def append(self, obj):
        self.data[self.index] = obj
        self.length = min(self.length + 1, self.max_length)
        self.index = (self.index + 1) % self.max_length

    def __getitem__(self, index):
        if index < 0 or index >= self.length:
            raise IndexError(f"replay buffer index out of range: {index}")
        return self.data[index]

    def __len__(self):
        return self.length

För att observera en episod i en omgivning för en viss policy:

In [58]:
def play_and_record_episode(model, env, replay_buffer, epsilon, seed=None):
    obs, _info = env.reset(seed=seed)
    total_rewards = 0
    model.eval()
    with torch.no_grad():
        while True:
            action = choose_dqn_action(model, obs, epsilon)
            next_obs, reward, done, truncated, _info = env.step(action)
            experience = (obs, action, reward, next_obs, done, truncated)
            replay_buffer.append(experience)
            total_rewards += reward
            if done or truncated:
                return total_rewards
            obs = next_obs

Ett träningssteg:

In [ ]:
# 'criterion' är loss-funktionen
def dqn_training_step(model, optimizer, criterion, replay_buffer, batch_size,
                      discount_factor):
    experiences = sample_experiences(replay_buffer, batch_size)
    state, action, reward, next_state, done, truncated = experiences
    with torch.inference_mode():
        next_Q_value = model(next_state)

    max_next_Q_value, _ = next_Q_value.max(dim=1)
    running = (~(done | truncated)).float()  # 0 slut, 1 pågående
    # vad vi 'borde' få, enligt nätverket:
    target_Q_value = reward + running * discount_factor * max_next_Q_value
    all_Q_values = model(state)
    Q_value = all_Q_values.gather(dim=1, index=action.unsqueeze(1))
    loss = criterion(Q_value, target_Q_value.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

En _Double DQN_ använder två nätverk, ett för förutsägelserna över erfarenheterna (next_Q_value ovan) och ett annat för nuvarande observationer (all_Q_values). Det tidigare nätverket lär sig i varje steg, men det andra uppdateras enligt något fördefinerat villkor genom att helt enkelt kopiera det första nätverkets vikter. Detta gör metoden mer stabil. Snart kommer vi se varför det behövs.

Att implementera en DDQN utifrån koden i denna notebook är en utmärkt övning!

Notera hur epsilon ändras under körningen. Detta motsvarar en _adaptive learning rate_, men minns att effekten är att agenten tar slumpmässiga handlingar med $\epsilon$ sannoliket!

In [ ]:
from collections import deque
import gymnasium as gym

def train_dqn(model, env, replay_buffer, optimizer, criterion, n_episodes=800,
              warmup=30, batch_size=32, discount_factor=0.95):
    totals = []
    for episode in range(n_episodes):
        epsilon = max(1 - episode / 500, 0.01)
        seed = torch.randint(0, 2**32, size=()).item()
        total_rewards = play_and_record_episode(model, env, replay_buffer,
                                                epsilon, seed=seed)
        print(f"\rEpisode: {episode + 1}, Rewards: {total_rewards}", end=" ")
        totals.append(total_rewards)
        if episode >= warmup:
            dqn_training_step(model, optimizer, criterion, replay_buffer,
                              batch_size, discount_factor)
    return totals


env = gym.make("CartPole-v1", render_mode="rgb_array", max_episode_steps=max_episode_steps)

torch.manual_seed(42)

dqn = DQN()
optimizer = torch.optim.NAdam(dqn.parameters(), lr=0.03)
mse = nn.MSELoss()
replay_buffer = deque(maxlen=100_000)
totals = train_dqn(dqn, env, replay_buffer, optimizer, mse)

Episode: 1000, Rewards: 1000.0 

In [ ]:
torch.manual_seed(42)
dqn.eval()

def dqn_policy(obs):
    with torch.no_grad():
        return choose_dqn_action(dqn, obs)

show_one_episode(dqn_policy, seed=42)

Failed? False
